# Video Processing

In [ ]:
import pandas as pd
import cv2

import 001_P48.csv

In [ ]:
csv_file_path = "/Users/yuqiliang/Documents/Python/SSID_IVR_Study/001_P48.csv"  
Points = pd.read_csv(csv_file_path, skiprows=lambda x: x < 26)

find the 'StartMedia' timestamp

In [ ]:
row = df[df['SlideEvent'] == 'StartMedia'] 
timestamp_diff = row['Timestamp'].values[0] 

## Overlay

In [ ]:

# read the video
input_video_path = "/Users/yuqiliang/Documents/Python/SSID_IVR_Study/1min_30.mp4"
video = cv2.VideoCapture(input_video_path)
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Print
print("FPS:", fps)
print("Total frames:", total_frames)
print(f"width: {width} ")
print(f"height: {height} ")

# VideoWriter mp4-mp4v format
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = "/Users/yuqiliang/Documents/Python/SSID_IVR_Study/output_01.mp4"
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# clean the NaN in columns
Points = Points.dropna(subset=['Gaze X', 'Gaze Y'])

# - timestamp_diff
Points['Timestamp'] = Points['Timestamp'] - timestamp_diff

# loop video frames
current_point_index = 0
for frame_index in range(total_frames):
    ret, frame = video.read()
    if not ret:
        break
    
    # time of the frame in ms
    # Convert frame index to ms
    current_time = frame_index / fps * 1000 
    
    # check if the time has reached the next timestamp
    while (current_point_index < len(Points) - 1 and
           Points['Timestamp'].iloc[current_point_index + 1] <= current_time):
        current_point_index += 1 #index=index+1

    # get the coordinates corresponding to the current timestamp
    if current_point_index < len(Points):
        row = Points.iloc[current_point_index]
        x, y = int(row['Gaze X']), int(row['Gaze Y'])
        # draw a yellow circle
        cv2.circle(frame, (x, y), 50, (0, 255, 255), -1) 

    out.write(frame) 

# Release resources
video.release()
out.release()
cv2.destroyAllWindows()

## Chop